In [1]:
## load useful libraries
suppressWarnings(suppressPackageStartupMessages(library(rhdf5)))
suppressWarnings(suppressPackageStartupMessages(library(qvalue)))
suppressWarnings(suppressPackageStartupMessages(library(dplyr)))
suppressWarnings(suppressPackageStartupMessages(library(cowplot)))
suppressWarnings(suppressPackageStartupMessages(library(tidyverse)))
suppressWarnings(suppressPackageStartupMessages(library(ggbeeswarm)))
suppressWarnings(suppressPackageStartupMessages(library(ggthemes)))
# suppressWarnings(suppressPackageStartupMessages(library(scater)))
suppressWarnings(suppressPackageStartupMessages(library(SingleCellExperiment)))
suppressWarnings(suppressPackageStartupMessages(library(edgeR)))
suppressWarnings(suppressPackageStartupMessages(library(limma)))
suppressWarnings(suppressPackageStartupMessages(library(ggplot2)))
suppressWarnings(suppressPackageStartupMessages(library(ISLR)))
suppressWarnings(suppressPackageStartupMessages(library(GGally)))
# suppressWarnings(suppressPackageStartupMessages(library(scran)))
suppressWarnings(suppressPackageStartupMessages(library(ComplexHeatmap)))
suppressWarnings(suppressPackageStartupMessages(library(circlize)))

In [2]:
snp_annos_folder = "/hps/nobackup/hipsci/scratch/genotypes/snp_annotation/renamed/"

In [3]:
SplitGeneInfo <- function(res){
    res$ensembl_gene_id = gsub("*_.*","",res$feature)
    res$gene_name = gsub(".*_","",res$feature)
    res
}

In [4]:
GetSnpRsids <- function(chrom){
    snp_anno = read.csv(paste0(snp_annos_folder,"chr",chrom,"_snps.txt"), sep = "\t")[,c("Variant.name",
                                                                                     "Chromosome.scaffold.name",
                                                                            "Chromosome.scaffold.position.start..bp.",
                                                                            "Chromosome.scaffold.position.end..bp.")]
    colnames(snp_anno) = c("snp_rsid", "chrom", "pos", "pos2")
    snp_anno[snp_anno$pos == snp_anno$pos2,c("snp_rsid", "chrom", "pos")]
}

In [26]:
AddSnpRsids <- function(res){
    res1 = res[res$chrom == 1,]
    snps = GetSnpRsids(chrom = 1)
    res1 = left_join(res1,snps)
    for (i in 2:22){
        res_cur = res[res$chrom == i,]
        snps = GetSnpRsids(chrom = i)
        res_cur = left_join(res_cur,snps, by = c("chrom", "pos"))
        res1 = rbind(res1,res_cur)
    }
    res1
}

In [16]:
mydir = "/hps/nobackup/hipsci/scratch/singlecell_endodiff/results_files_20181031/eQTL_results_dfs/"

In [7]:
### stages

In [14]:
ips.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/ips/leads.csv", row.names = 1)
ips.leads = SplitGeneInfo(ips.leads)
ips.leads = AddSnpRsids(ips.leads)
head(ips.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,alt_allele,ensembl_gene_id,gene_name,snp_rsid
-0.9842330,0.10208734,2.113107e-18,5.361668e-22,1_24364016_T_C,ENSG00000189266_PNRC2,8.186493e-16,1,24364016,T,C,ENSG00000189266,PNRC2,rs61773530
1.1455146,0.11663421,2.723503e-16,9.104331e-23,1_171662929_T_C,ENSG00000117533_VAMP4,6.273722e-14,1,171662929,T,C,ENSG00000117533,VAMP4,NA
-0.9975982,0.10269964,9.337197e-15,2.634728e-22,1_75189847_T_C,ENSG00000116791_CRYZ,1.624125e-12,1,75189847,T,C,ENSG00000116791,CRYZ,rs1424671
-0.7300103,0.08154161,2.505289e-14,3.471687e-19,1_19911802_T_C,ENSG00000173436_MINOS1,4.106328e-12,1,19911802,T,C,ENSG00000173436,MINOS1,rs2792036
-0.4522483,0.05871334,2.855039e-12,1.332738e-14,1_89447978_T_A,ENSG00000137944_CCBL2,2.967545e-10,1,89447978,T,A,ENSG00000137944,CCBL2,rs10754258
-0.6477022,0.08402685,2.798272e-11,1.275279e-14,1_162726281_A_C,ENSG00000132196_HSD17B7,2.409094e-09,1,162726281,A,C,ENSG00000132196,HSD17B7,rs2684866


In [18]:
ips.leads.filename = paste0(mydir,"ips.leads.txt")
write.table(ips.leads, ips.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [21]:
# mesendo.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/mesendo_est_June20/leads.csv", row.names = 1)
# mesendo.leads = SplitGeneInfo(mesendo.leads)
# mesendo.leads = AddSnpRsids(mesendo.leads)
head(mesendo.leads)

beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,⋯,permutation_3,permutation_4,permutation_5,permutation_6,permutation_7,permutation_8,permutation_9,ensembl_gene_id,gene_name,snp_rsid
-1.0268717,0.10360882,4.852290e-23,3.727273e-23,1_24364016_T_C,ENSG00000189266_PNRC2,3.186781e-20,1,24364016,T,⋯,0.2431465,0.95948844,0.95584859,0.21782557,0.7011236,0.60835092,0.01993654,ENSG00000189266,PNRC2,rs61773530
-1.0355745,0.10365492,2.889539e-22,1.675701e-23,1_75189847_T_C,ENSG00000116791_CRYZ,1.626624e-19,1,75189847,T,⋯,0.9417918,0.08161423,0.88777330,0.93693458,0.2180240,0.14445355,0.14499708,ENSG00000116791,CRYZ,rs1424671
-0.6957178,0.07540540,2.764506e-17,2.799692e-20,1_39465139_A_C,ENSG00000168653_NDUFS5,7.781193e-15,1,39465139,A,⋯,0.5668985,0.93862277,0.62489552,0.87066206,0.1522135,0.67846606,0.21201485,ENSG00000168653,NDUFS5,rs11205522
-0.7409688,0.09815311,3.466158e-15,4.382373e-14,1_108739849_C_T,ENSG00000085491_SLC25A24,6.504078e-13,1,108739849,C,⋯,0.0207375,0.72430557,0.17900311,0.08055645,0.4630297,0.20818287,0.77572728,ENSG00000085491,SLC25A24,rs676166
-0.8706880,0.10360809,1.616229e-13,4.327428e-17,1_75189847_T_C,ENSG00000162623_TYW3,2.493608e-11,1,75189847,T,⋯,0.1125812,0.40372242,0.86585812,0.43187795,0.6991340,0.54130774,0.66269092,ENSG00000162623,TYW3,rs1424671
-0.9432526,0.10400597,4.973804e-13,1.198767e-19,1_19911802_T_C,ENSG00000173436_MINOS1,6.322424e-11,1,19911802,T,⋯,0.1227212,0.30064527,0.05977199,0.97591189,0.6105252,0.04437388,0.23966650,ENSG00000173436,MINOS1,rs2792036


In [22]:
mesendo.leads.filename = paste0(mydir,"mesendo.leads.txt")
write.table(mesendo.leads, mesendo.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [23]:
defendo.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/defendo_est_June20/leads.csv", row.names = 1)
defendo.leads = SplitGeneInfo(defendo.leads)
defendo.leads = AddSnpRsids(defendo.leads)
head(defendo.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,⋯,permutation_3,permutation_4,permutation_5,permutation_6,permutation_7,permutation_8,permutation_9,ensembl_gene_id,gene_name,snp_rsid
-0.9122687,0.08767589,8.733500e-32,2.352239e-25,1_39465139_A_C,ENSG00000168653_NDUFS5,1.979745e-28,1,39465139,A,⋯,0.73907615,0.7649246,0.4915471,0.3066058,0.24123457,0.1187501,0.2318245,ENSG00000168653,NDUFS5,rs11205522
-1.0292186,0.10839444,1.491397e-18,2.199496e-21,1_75189847_T_C,ENSG00000116791_CRYZ,9.015362e-16,1,75189847,T,⋯,0.95221972,0.2188547,0.5712642,0.3292014,0.53891598,0.4024206,0.6674582,ENSG00000116791,CRYZ,rs1424671
-0.8776222,0.09583309,5.576915e-12,5.295623e-20,1_24364016_T_C,ENSG00000189266_PNRC2,9.029985e-10,1,24364016,T,⋯,0.09031542,0.7494634,0.1307345,0.8572970,0.66410981,0.3447975,0.8668023,ENSG00000189266,PNRC2,rs61773530
-0.4649860,0.07188726,1.598049e-09,9.913315e-11,1_165599649_C_T,ENSG00000143198_MGST3,1.628100e-07,1,165599649,C,⋯,0.77081351,0.2534301,0.6816288,0.8771795,0.72715659,0.5779859,0.5962529,ENSG00000143198,MGST3,rs10918223
0.7671610,0.11228109,4.333594e-09,8.344512e-12,1_171651692_T_C,ENSG00000117533_VAMP4,3.929428e-07,1,171651692,T,⋯,0.49069360,0.1893900,0.8999207,0.8722565,0.38342878,0.6900384,0.6204222,ENSG00000117533,VAMP4,rs188447967
-0.8383388,0.11259674,1.288480e-08,9.657903e-14,1_75189847_T_C,ENSG00000162623_TYW3,1.033904e-06,1,75189847,T,⋯,0.43655165,0.4449808,0.4177315,0.4582476,0.02139354,0.6093945,0.7476584,ENSG00000162623,TYW3,rs1424671


In [24]:
defendo.leads.filename = paste0(mydir,"defendo.leads.txt")
write.table(defendo.leads, defendo.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [ ]:
### days

In [25]:
day0.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/day0/all_expts/leads.csv", row.names = 1)
day0.leads = SplitGeneInfo(day0.leads)
day0.leads = AddSnpRsids(day0.leads)
head(day0.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,⋯,permutation_3,permutation_4,permutation_5,permutation_6,permutation_7,permutation_8,permutation_9,ensembl_gene_id,gene_name,snp_rsid
-0.9269966,0.09116613,6.801554e-21,2.749088e-24,1_24364016_T_C,ENSG00000189266_PNRC2,3.853176e-18,1,24364016,T,⋯,0.30905805,0.2992134,0.8111852,0.10595850,0.1529934,0.97220644,0.44658741,ENSG00000189266,PNRC2,rs61773530
-0.9866419,0.10132891,3.185062e-16,2.096053e-22,1_75189847_T_C,ENSG00000116791_CRYZ,5.874733e-14,1,75189847,T,⋯,0.01707314,0.6591003,0.3334245,0.10354072,0.3355486,0.98578247,0.35181971,ENSG00000116791,CRYZ,rs1424671
-0.3888929,0.04712030,4.870883e-15,1.542185e-16,1_89373371_T_C,ENSG00000137944_CCBL2,6.898559e-13,1,89373371,T,⋯,0.34827366,0.9931216,0.2162904,0.08175724,0.4427933,0.44144835,0.59110954,ENSG00000137944,CCBL2,rs10801690
1.0605896,0.10870507,6.433826e-15,1.728897e-22,1_171667160_G_A,ENSG00000117533_VAMP4,8.648803e-13,1,171667160,G,⋯,0.73328020,0.1180319,0.7829828,0.01499698,0.3892203,0.57258671,0.59450470,ENSG00000117533,VAMP4,rs11578179
-0.7868997,0.08318238,4.012253e-14,3.081419e-21,1_108739849_C_T,ENSG00000085491_SLC25A24,4.749547e-12,1,108739849,C,⋯,0.92646861,0.3740020,0.5832400,0.75489798,0.7363466,0.59336176,0.09158537,ENSG00000085491,SLC25A24,rs676166
-0.6967376,0.07465868,2.833927e-13,1.035920e-20,1_19911802_T_C,ENSG00000173436_MINOS1,2.957425e-11,1,19911802,T,⋯,0.44634492,0.1103569,0.2687007,0.36388378,0.5396158,0.09768087,0.03710821,ENSG00000173436,MINOS1,rs2792036


In [27]:
day0.leads.filename = paste0(mydir,"day0.leads.txt")
write.table(day0.leads, day0.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [28]:
day1.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/day1/leads.csv", row.names = 1)
day1.leads = SplitGeneInfo(day1.leads)
day1.leads = AddSnpRsids(day1.leads)
head(day1.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,alt_allele,ensembl_gene_id,gene_name,snp_rsid
-1.0107911,0.09562235,1.302604e-27,4.076232e-26,1_24364016_T_C,ENSG00000189266_PNRC2,3.785642e-24,1,24364016,T,C,ENSG00000189266,PNRC2,rs61773530
-1.0570988,0.10238749,2.785332e-24,5.460716e-25,1_75214390_G_C,ENSG00000116791_CRYZ,2.229531e-21,1,75214390,G,C,ENSG00000116791,CRYZ,rs749690
-0.8442254,0.08368277,2.169530e-19,6.217371e-24,1_19911802_T_C,ENSG00000173436_MINOS1,9.060570e-17,1,19911802,T,C,ENSG00000173436,MINOS1,rs2792036
-0.6699340,0.06688676,4.140256e-18,1.297189e-23,1_39465139_A_C,ENSG00000168653_NDUFS5,1.590761e-15,1,39465139,A,C,ENSG00000168653,NDUFS5,rs11205522
-0.8322924,0.09416837,2.785441e-14,9.714873e-19,1_21795388_A_G,ENSG00000142794_NBPF3,5.246160e-12,1,21795388,A,G,ENSG00000142794,NBPF3,rs1827293
1.0983003,0.12207278,7.814221e-14,2.317699e-19,1_171728983_G_A,ENSG00000117533_VAMP4,1.316827e-11,1,171728983,G,A,ENSG00000117533,VAMP4,rs2421911


In [29]:
day1.leads.filename = paste0(mydir,"day1.leads.txt")
write.table(day1.leads, day1.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [31]:
day3.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/day3/leads.csv", row.names = 1)
day3.leads = SplitGeneInfo(day3.leads)
day3.leads = AddSnpRsids(day3.leads)
head(day3.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,alt_allele,ensembl_gene_id,gene_name,snp_rsid
-0.8538697,0.07727158,6.639585e-24,2.186208e-28,1_39465139_A_C,ENSG00000168653_NDUFS5,1.307378e-20,1,39465139,A,C,ENSG00000168653,NDUFS5,rs11205522
-1.0511882,0.09949660,2.312070e-19,4.326639e-26,1_75189847_T_C,ENSG00000116791_CRYZ,2.845385e-16,1,75189847,T,C,ENSG00000116791,CRYZ,rs1424671
-0.8375996,0.09190622,3.984506e-13,7.966842e-20,1_24364016_T_C,ENSG00000189266_PNRC2,1.352716e-10,1,24364016,T,C,ENSG00000189266,PNRC2,rs61773530
-0.7832241,0.10073623,1.425512e-08,7.544810e-15,1_19911802_T_C,ENSG00000173436_MINOS1,2.004947e-06,1,19911802,T,C,ENSG00000173436,MINOS1,rs2792036
-0.8212972,0.11030539,2.539283e-08,9.645607e-14,1_75189847_T_C,ENSG00000162623_TYW3,3.378389e-06,1,75189847,T,C,ENSG00000162623,TYW3,rs1424671
0.4141181,0.06700404,1.940306e-06,6.390095e-10,1_234511503_A_G,ENSG00000168275_COA6,1.471583e-04,1,234511503,A,G,ENSG00000168275,COA6,rs7523348


In [32]:
day3.leads.filename = paste0(mydir,"day3.leads.txt")
write.table(day3.leads, day3.leads.filename, sep = "\t", quote = F, row.names = FALSE)

In [ ]:
### bulk ips (day0)

In [33]:
bulk_ips.leads = read.csv("/nfs/leia/research/stegle/acuomo/mean/bulk_ipsc/leads.csv", row.names = 1)
bulk_ips.leads = SplitGeneInfo(bulk_ips.leads)
bulk_ips.leads = AddSnpRsids(bulk_ips.leads)
head(bulk_ips.leads)

Joining, by = c("chrom", "pos")


beta,beta_se,empirical_feature_p_value,p_value,snp_id,feature,global_corr_p_value,chrom,pos,ref_allele,alt_allele,ensembl_gene_id,gene_name,snp_rsid
-0.6115009,0.05229288,2.003786e-22,1.371617e-31,1_46806242_G_T,ENSG00000117481_NSUN4,2.616424e-20,1,46806242,G,T,ENSG00000117481,NSUN4,rs67545510
-1.2670836,0.09204958,8.186464e-22,4.125709e-43,1_146628479_C_G,ENSG00000131791_PRKAB2,9.620458e-20,1,146628479,C,G,ENSG00000131791,PRKAB2,rs11584787
-1.0148197,0.08978129,8.362668e-21,1.264597e-29,1_146735170_G_A,ENSG00000131778_CHD1L,8.934115e-19,1,146735170,G,A,ENSG00000131778,CHD1L,rs2883432
-1.0018160,0.09248270,4.149085e-20,2.415499e-27,1_24364016_T_C,ENSG00000189266_PNRC2,4.132089e-18,1,24364016,T,C,ENSG00000189266,PNRC2,rs61773530
0.6451078,0.06380361,1.158593e-19,4.946117e-24,1_78392446_G_A,ENSG00000162616_DNAJB4,1.063703e-17,1,78392446,G,A,ENSG00000162616,DNAJB4,rs1166698
-0.4794575,0.04201419,4.071285e-19,3.650944e-30,1_179892028_G_A,ENSG00000143337_TOR1AIP1,3.517969e-17,1,179892028,G,A,ENSG00000143337,TOR1AIP1,rs644412


In [34]:
bulk_ips.leads.filename = paste0(mydir,"bulk.ips.leads.txt")
write.table(bulk_ips.leads, bulk_ips.leads.filename, sep = "\t", quote = F, row.names = FALSE)